## **Assignment: Ionosphere Data Problem**

Dataset Description:
This radar data was collected by a system in Goose Bay, Labrador. This system consists of a phased array of 16 high-frequency antennas with a total transmitted power on the order of 6.4 kilowatts. See the paper for more details. The targets were free electrons in the ionosphere. "Good" radar returns are those showing evidence of some type of structure in the ionosphere. "Bad" returns are those that do not; their signals pass through the ionosphere.

Received signals were processed using an autocorrelation function whose arguments are the time of a pulse and the pulse number. There were 17 pulse numbers for the Goose Bay system. Instances in this databse are described by 2 attributes per pulse number, corresponding to the complex values returned by the function resulting from the complex electromagnetic signal.




In [586]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [587]:
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline

In [588]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler 
import math
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD

### Load Data

In [589]:
data = pd.read_csv("ionosphere_data.csv")
data.head(10)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,label
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
5,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,0.03786,-0.06302,0.00000,0.00000,-0.04572,-0.15540,-0.00343,-0.10196,-0.11575,-0.05414,0.01838,0.03669,0.01519,0.00888,0.03513,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,b
6,1,0,0.97588,-0.10602,0.94601,-0.20800,0.92806,-0.28350,0.85996,-0.27342,0.79766,-0.47929,0.78225,-0.50764,0.74628,-0.61436,0.57945,-0.68086,0.37852,-0.73641,0.36324,-0.76562,0.31898,-0.79753,0.22792,-0.81634,0.13659,-0.82510,0.04606,-0.82395,-0.04262,-0.81318,-0.13832,-0.80975,g
7,0,0,0.00000,0.00000,0.00000,0.00000,1.00000,-1.00000,0.00000,0.00000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,1.00000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000,0.00000,b
8,1,0,0.96355,-0.07198,1.00000,-0.14333,1.00000,-0.21313,1.00000,-0.36174,0.92570,-0.43569,0.94510,-0.40668,0.90392,-0.46381,0.98305,-0.35257,0.84537,-0.66020,0.75346,-0.60589,0.69637,-0.64225,0.85106,-0.65440,0.57577,-0.69712,0.25435,-0.63919,0.45114,-0.72779,0.38895,-0.73420,g
9,1,0,-0.01864,-0.08459,0.00000,0.00000,0.00000,0.00000,0.11470,-0.26810,-0.45663,-0.38172,0.00000,0.00000,-0.33656,0.38602,-0.37133,0.15018,0.63728,0.22115,0.00000,0.00000,0.00000,0.00000,-0.14803,-0.01326,0.20645,-0.02294,0.00000,0.00000,0.16595,0.24086,-0.08208,0.38065,b


In [590]:
data.shape

(351, 35)

### Check Missing Values ( If Exist ; Fill each record with mean of its feature )

In [591]:
#no null values exist in the data. 
data.isnull().sum() 

feature1     0
feature2     0
feature3     0
feature4     0
feature5     0
feature6     0
feature7     0
feature8     0
feature9     0
feature10    0
feature11    0
feature12    0
feature13    0
feature14    0
feature15    0
feature16    0
feature17    0
feature18    0
feature19    0
feature20    0
feature21    0
feature22    0
feature23    0
feature24    0
feature25    0
feature26    0
feature27    0
feature28    0
feature29    0
feature30    0
feature31    0
feature32    0
feature33    0
feature34    0
label        0
dtype: int64

### Standardized the Input Variables. Hint: Centeralized the data


In [592]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature1   351 non-null    int64  
 1   feature2   351 non-null    int64  
 2   feature3   351 non-null    float64
 3   feature4   351 non-null    float64
 4   feature5   351 non-null    float64
 5   feature6   351 non-null    float64
 6   feature7   351 non-null    float64
 7   feature8   351 non-null    float64
 8   feature9   351 non-null    float64
 9   feature10  351 non-null    float64
 10  feature11  351 non-null    float64
 11  feature12  351 non-null    float64
 12  feature13  351 non-null    float64
 13  feature14  351 non-null    float64
 14  feature15  351 non-null    float64
 15  feature16  351 non-null    float64
 16  feature17  351 non-null    float64
 17  feature18  351 non-null    float64
 18  feature19  351 non-null    float64
 19  feature20  351 non-null    float64
 20  feature21 

In [593]:
#Splitting numeric float values for standadization
x = data.iloc[:,:-1].values
x

array([[ 1.     ,  0.     ,  0.99539, ..., -0.54487,  0.18641, -0.453  ],
       [ 1.     ,  0.     ,  1.     , ..., -0.06288, -0.13738, -0.02447],
       [ 1.     ,  0.     ,  1.     , ..., -0.2418 ,  0.56045, -0.38238],
       ...,
       [ 1.     ,  0.     ,  0.94701, ...,  0.00442,  0.92697, -0.00577],
       [ 1.     ,  0.     ,  0.90608, ..., -0.03757,  0.87403, -0.16243],
       [ 1.     ,  0.     ,  0.8471 , ..., -0.06678,  0.85764, -0.06151]])

In [594]:
x.shape

(351, 34)

In [595]:
from numpy import asarray
from sklearn.preprocessing import StandardScaler
# define standard scaler
scaler = StandardScaler()
# transform data
X = scaler.fit_transform(x)
print(scaled)

[[ 0.34843328  0.          0.71237237 ... -1.05505394 -0.3122206
  -0.99959483]
 [ 0.34843328  0.          0.72164805 ... -0.11521328 -0.93260505
  -0.08328554]
 [ 0.34843328  0.          0.72164805 ... -0.46409249  0.40444328
  -0.84859079]
 ...
 [ 0.34843328  0.          0.61502805 ...  0.01601615  1.10669878
  -0.04330004]
 [ 0.34843328  0.          0.53267371 ... -0.06586087  1.00526528
  -0.37828012]
 [ 0.34843328  0.          0.41400137 ... -0.12281796  0.9738619
  -0.16248675]]


### Encoding Label feature

In [596]:
#Splitting categorical values for encoding
y = data.iloc[:,-1].values
y

array(['g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b',
       'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b',
       'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b',
       'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b',
       'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b',
       'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g',
       'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b',
       'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b', 'g', 'b

In [597]:
y.shape

(351,)

In [598]:
encoder = LabelEncoder() 
encoder.fit(y) 
Y = encoder.transform(y)

### Split into 60 and 40 ratio.

In [599]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.40,random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(210, 34) (210,)
(141, 34) (141,)


### Model : 1 hidden layers including 16 unit.


In [600]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1))

### Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

In [601]:
# Compile model
     
learning_rate = 0.1 
decay_rate = learning_rate / epochs 
momentum = 0.8 
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(loss= "binary_crossentropy" , optimizer=sgd, metrics=[ "accuracy" ]) 

### Train the Model with Epochs (100).


In [602]:
epochs = 100 
model.fit(x_train, y_train, epochs=epochs, batch_size=64)

Epoch 1/100
4/4 [==============================] - 0s 3ms/step - loss: 7.7373 - accuracy: 0.3981
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 5.2135 - accuracy: 0.6283
Epoch 3/100
4/4 [==============================] - 0s 3ms/step - loss: 4.0784 - accuracy: 0.7056
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 3.3123 - accuracy: 0.7816
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 3.1054 - accuracy: 0.7938
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.9492 - accuracy: 0.7972
Epoch 7/100
4/4 [==============================] - 0s 2ms/step - loss: 2.5234 - accuracy: 0.8286
Epoch 8/100
4/4 [==============================] - 0s 3ms/step - loss: 2.3756 - accuracy: 0.8449
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1789 - accuracy: 0.8576
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 2.0905 - accuracy: 0.8633
Epoch 11/100
4/4 [===========

### Evaluation Step

In [603]:
test_loss, test_acc = model.evaluate(x_test, y_test) 
print('test_acc:', test_acc)

5/5 [==============================] - 0s 2ms/step - loss: 2.1853 - accuracy: 0.8511
test_acc: 0.8510638475418091


### If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

### Prediction

In [604]:
model.predict(x_test)

array([[   4.7836404 ],
       [   4.09692   ],
       [   6.532423  ],
       [   4.2703214 ],
       [   4.007483  ],
       [   4.075875  ],
       [   4.0529776 ],
       [   6.6756096 ],
       [   6.0885735 ],
       [ -14.665889  ],
       [  -1.0044374 ],
       [ -17.460411  ],
       [ -50.98668   ],
       [   3.9500651 ],
       [   4.038322  ],
       [   4.0673337 ],
       [   6.584284  ],
       [ -13.58837   ],
       [   3.9198446 ],
       [ -30.735905  ],
       [   4.061664  ],
       [ -26.255692  ],
       [ -74.006836  ],
       [   3.9259965 ],
       [   5.5353613 ],
       [   0.51592517],
       [ -16.936684  ],
       [ -23.635551  ],
       [ -15.830678  ],
       [   3.9955966 ],
       [   4.037066  ],
       [   4.0076594 ],
       [   4.1149683 ],
       [   4.0754504 ],
       [   6.2462645 ],
       [ -64.46727   ],
       [   5.6518345 ],
       [   4.0113955 ],
       [ -10.765924  ],
       [   6.486496  ],
       [   6.8904963 ],
       [   4.114